### Extraire les informations sur les équipes participant à la Premier League pour la saison 2024–2025.

In [116]:
import pandas as pd
import numpy as np
import os
import time

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=chrome_options)

driver.get("https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats")


- Créer un dossier (teams) pour stocker les données

In [126]:
raw_path = '../data/raw'
raw_teams_path = os.path.join(raw_path, 'teams')

os.makedirs(raw_teams_path, exist_ok=True)

- Selectionner tous les équipes:

In [ ]:
try:
    squads_table = driver.find_element(By.CSS_SELECTOR, '.stats_table.sortable')
except:
    print('Table Not found')
else:
    squads = squads_table.find_elements(By.CSS_SELECTOR, 'tbody > tr td[data-stat="team"] a')


- Pour chaque équipe, selectionner les information de chaque joueur

In [ ]:
for squad in squads:

    # Selectionner la premiére fenêtre
    driver.switch_to.window(driver.window_handles[0])

    squad_path = os.path.join(raw_teams_path, squad.text)

    os.makedirs(squad_path, exist_ok=True)

    # Ovrir le lien dans une nouvelle fenêtre
    squad_page_link = squad.get_attribute('href')
    driver.execute_script(f"window.open('{squad_page_link}', '_blank');")

    # Selectionner la deuxiéme fenêtre
    driver.switch_to.window(driver.window_handles[-1])

    time.sleep(3)
    
    # Selectionner les deux tables: joueurs et matches
    players_table, matches_table = driver.find_elements(By.CSS_SELECTOR, '.stats_table.sortable')[:2]

    # Selectionner les colonnes de table joueurs
    players_table_cols = players_table.find_elements(By.CSS_SELECTOR, 'thead > tr:last-child > *')[:16]
    players_table_cols = list(map(lambda x: x.text, players_table_cols))

    # Selectionner les lignes du tableau
    players_table_rows = players_table.find_elements(By.CSS_SELECTOR, 'tbody > tr:not([class])')

    details_list = []

    # Pour chaque joueur, stocker leur details 
    for row in players_table_rows:
        row_selector = row.find_elements(By.CSS_SELECTOR, '[data-stat]')[:16]

        player_details = list(map(lambda x: x.text, row_selector))

        details_list.append(player_details)

    # Transformer la liste en Dataframe
    players_dict = {}

    for i in range(len(players_table_cols)):
        players_dict[players_table_cols[i]] = [item[i] for item in details_list]

    players_df = pd.DataFrame(players_dict)

    # Les valeur sont sous forme (eg Egy), alors on garde la derniere valeur
    players_df['Nation'] = players_df['Nation'].apply(lambda x: x.split(' ')[-1])

    # Sauvegarder le tableau sous forme d'un fichier csv
    players_df.to_csv(squad_path + '/players.csv', index=False)

    # Selectionner les colonnes de table matches
    matches_table_cols = matches_table.find_elements(By.CSS_SELECTOR, 'thead > tr > th')[:18]
    matches_table_cols = list(map(lambda x: x.text, matches_table_cols))

    # Selectionner les lignes du tableau
    matches_table_rows = matches_table.find_elements(By.CSS_SELECTOR, 'tbody > tr:not([class])')

    details_list = []

    # Pour chaque joueur, stocker leur details 
    for row in matches_table_rows:
        row_selector = row.find_elements(By.CSS_SELECTOR, '[data-stat]')[:18]

        match_details = list(map(lambda x: x.text, row_selector))

        details_list.append(match_details)

    # Transformer la liste en Dataframe
    matches_dict = {}

    for i in range(len(matches_table_cols)):
        matches_dict[matches_table_cols[i]] = [item[i] for item in details_list]

    matches_df = pd.DataFrame(matches_dict)

    # Sauvegarder le tableau sous forme d'un fichier csv
    matches_df.to_csv(squad_path + '/matches.csv', index=False)

    # Fermer la fenêtre courante
    driver.close()

driver.quit()



